# Train Angle Regression Model

Script to train model on landmark data and camera metadata using Ridge regression. Purpose of model is to understand relationship between pixel location + camera elevation and angle between camera centerline and smoke in image. Pickles model and scaler into files locating_angle_mode.gzip and locating_scaler.gzip. Should be run whenever a new landmark is added to landmarks_manual.csv or an existing camera's metadata changes.

## Import Packages

In [1]:
import pandas as pd
import numpy as np

import sklearn.linear_model as lin_model

from haversine import haversine, Unit, inverse_haversine

import sklearn.model_selection as model
import sklearn.preprocessing as preprop
from sklearn.metrics import r2_score
import joblib
from ast import literal_eval

## Load Camera Metadata

In [3]:
df_cameras = pd.read_csv(f"../../../data/processed/camera_metadata_hpwren.csv")
df_cameras = df_cameras.dropna(subset=['x_resolution', 'y_resolution', 'center_angle'])

In [4]:
df_cameras.head()

,camera_id,direction,camera_name,camera_abbrev,image_id,long,lat,elevation,geometry.type,geometry.coordinates,x_resolution,y_resolution,center_lat,center_long,center_angle,properties.description.url,intersections
1,hpwren1_north,north,Big Black Mountain,bm,bm-n-mobo-c,-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,33.181599,-116.807554,-0.024816,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,"[('bl', 'north'), ('bl', 'east'), ('bh', 'nort..."
2,hpwren1_east,east,Big Black Mountain,bm,bm-e-mobo-c,-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,33.158781,-116.790230,-0.064085,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,"[('bh', 'east'), ('bh', 'south'), ('cp', 'nort..."
3,hpwren1_south,south,Big Black Mountain,bm,bm-s-mobo-c,-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,33.157932,-116.807962,0.065022,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,"[('bl', 'east'), ('bl', 'south'), ('bh', 'sout..."
4,hpwren1_west,west,Big Black Mountain,bm,bm-w-mobo-c,-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,33.159091,-116.858706,0.016519,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,"[('bl', 'north'), ('bl', 'east'), ('bl', 'sout..."
5,hpwren2_north,north,Black Mountain,bl,bl-n-mobo-c,-117.116480,32.981417,1600.0,Point,"[-117.12, 32.98, 1600]",2048.0,1536.0,32.981788,-117.116520,0.108221,http://hpwren.ucsd.edu/cameras/BL.html,"[('bm', 'north'), ('bm', 'west'), ('bh', 'east..."


## Load Landmark Data

In [5]:
df_lm = pd.read_csv('../../../data/raw/landmarks_manual.csv')
df_lm = df_lm.rename(columns={'lat': 'lm_lat', 'long': 'lm_long'})

## Join Datasets

In [6]:
def find_angle(ax, ay, bx, by, cx, cy):
    a = np.array([ax, ay])
    b = np.array([bx, by])
    c = np.array([cx, cy])

    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return angle

In [7]:
df_merged = df_cameras.merge(df_lm, left_on=['camera_abbrev', 'direction'], right_on=['camera_abbrev', 'direction'], how='inner')
df_merged = df_merged[['camera_abbrev', 'direction', 'landmark', 'lat', 'long', 'center_lat', 'center_long', 'lm_lat', 'lm_long', 'x_pixel', 'y_pixel', 'x_res', 'y_res', 'elevation', 'distance', 'intersection', 'center_angle']]
df_merged['angle'] = df_merged.apply(lambda x: find_angle(x['center_long'], x['center_lat'], x['long'], x['lat'], x['lm_long'], x['lm_lat']), axis=1)

## Convert Data and Train Model

In [8]:
def find_x_ratio(pix, res):
    return abs(pix-(res/2))/(res/2)

def find_y_ratio(pix, res):
    return abs(res-pix)/(res)

df_merged['x_ratio'] = df_merged.apply(lambda x: find_x_ratio(x['x_pixel'], x['x_res']), axis=1)
df_merged['y_ratio'] = df_merged.apply(lambda x: find_y_ratio(x['y_pixel'], x['y_res']), axis=1)

In [10]:
X_test = df_merged[['x_ratio', 'y_ratio', 'elevation']].loc[df_merged['intersection'] == 1]
X_test['elevation'] = X_test['elevation'] - np.median(X_test['elevation'])
y_test = df_merged['angle'].loc[df_merged['intersection'] == 1]

X_train = df_merged[['x_ratio', 'y_ratio', 'elevation']].loc[df_merged['intersection'] == 0]
X_train['elevation'] = X_train['elevation'] - np.median(X_train['elevation'])
y_train = df_merged['angle'].loc[df_merged['intersection'] == 0]

#Scale Data
scaler = preprop.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

X_test = scaler.transform(X_test)

#Fit Model
model = lin_model.Ridge(alpha=11.5, max_iter=1000)#, selection='random')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("R2 Score: ", r2_score(y_test, y_pred))


R2 Score:  0.9136320621646693


## Pickle/Store Model

In [11]:
joblib.dump(model, '../pickled_files/locating_angle_model.gz')
joblib.dump(scaler, '../pickled_files/locating_scaler.gz')
joblib.dump(df_merged, '../pickled_files/test_df.gz')

['../pickled_files/test_df.gz']